In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [18]:
data = pd.read_csv('02balanced.csv')
data.sample(5)

,Name,Date,Sex_F,Appetite,SleepInd,StSpirit,pEffort,uaI,uaI(-1),DeltaWt (%),DeltaHrtwo,FatigInd
268,F_m,2020-03-04,0,4.0,-4.0,4.0,6.0,2.76,2.44,-0.012346,22.0,Risk/Caution
101,I_f,2019-12-22,1,6.0,-5.0,5.0,1.0,1.00,1.00,-0.002506,0.0,Low/Minimal
66,V_f,2024-12-24,1,5.0,-5.0,3.0,1.0,0.00,1.00,-0.006981,0.0,Low/Minimal
96,I_f,2019-12-23,1,6.0,-5.0,5.0,4.0,1.00,1.00,-0.005025,0.0,Low/Minimal
182,I_f,2020-01-27,1,6.0,-5.0,5.0,6.0,2.02,0.00,0.012407,24.0,Optimal


In [19]:
def standardize_numeric_columns(df):
    # Separate numeric and non-numeric columns
    numeric_cols = ['Appetite', 'SleepInd', 'StSpirit', 'pEffort',
                    'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']

    # Initialize StandardScaler
    scaler = StandardScaler()

    # Standardize only the numeric columns
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Display the resulting DataFrame
    return df 

df = standardize_numeric_columns(data)

In [20]:
def logistical_regression(df, sex="all", only_accuracy=False):
       if sex=="all":
              filtered_df = df
       elif sex=="F":
              filtered_df = df[df["Sex_F"] > 0.5]
       elif sex=="M":
              filtered_df = df[df["Sex_F"] < 0.5]
       else:
              raise ValueError("Invalid")


       X = filtered_df[['Sex_F', 'Appetite', 'SleepInd', 'StSpirit', 'pEffort',
              'uaI', 'uaI(-1)', 'DeltaWt (%)', 'DeltaHrtwo']]
       y = filtered_df['FatigInd']

       X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)
       # Initialize the model
       model = LogisticRegression()

       # Train the model
       model.fit(X_train, y_train)

       # Make predictions
       y_pred = model.predict(X_test)

       # Calculate accuracy
       accuracy = accuracy_score(y_test, y_pred)
       print(f'Accuracy: {accuracy}')

       # Display confusion matrix
       print("Confusion Matrix:")
       print(confusion_matrix(y_test, y_pred))

       if not only_accuracy:
              # Classification report for precision, recall, and F1-score
              print("Classification Report:")
              print(classification_report(y_test, y_pred))
              # Display coefficients of the model (for each feature)
              print("Model Coefficients:", model.coef_)
              print("Intercept:", model.intercept_)

logistical_regression(df, sex="all", only_accuracy=True)
logistical_regression(df, sex="F", only_accuracy=True)
logistical_regression(df, sex="M", only_accuracy=True)

Accuracy: 0.704225352112676
Confusion Matrix:
[[19  4  1]
 [ 6 13  6]
 [ 0  4 18]]
Accuracy: 0.5277777777777778
Confusion Matrix:
[[12  2  1]
 [ 5  4  5]
 [ 1  3  3]]
Accuracy: 0.5833333333333334
Confusion Matrix:
[[10  2  3]
 [ 4  6  4]
 [ 0  2  5]]
